In [97]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
from scipy.sparse import vstack

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8433 candidates
Dev set:	920 candidates
Test set:	4683 candidates


In [120]:
# larger training set because this is the hand labeled model
dev.extend(train[:1000])

In [99]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)

In [121]:
combined = vstack((L_gold_dev, L_gold_train[:1000]))

In [125]:
combined[combined == -1] = 0
y = np.append(1-combined, combined,axis=1)

In [126]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':              0.01,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        20,
    'dropout':         0.5,
    'rebalance':       0.25,
    'print_freq':      5,
    'seed':            1701
}

lstm = LSTM(n_threads=None)
lstm.train(dev, y, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=1675  #epochs=20  batch size=64
[LSTM] Epoch 1 (4.79s)	Average loss=0.655357
[LSTM] Epoch 6 (29.58s)	Average loss=0.270034
[LSTM] Epoch 11 (54.26s)	Average loss=0.156114
[LSTM] Epoch 16 (79.05s)	Average loss=0.093922
[LSTM] Epoch 20 (98.78s)	Average loss=0.084508
[LSTM] Training done (98.78s)


In [102]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

AnnotatorLabels created: 0


<4683x1 sparse matrix of type '<class 'numpy.int64'>'
	with 4683 stored elements in Compressed Sparse Row format>

In [127]:
lstm.score(test, L_gold_test) # prec, rec, f1 I think

(0.4670886075949367, 0.48745046235138706, 0.47705235940530055)

In [65]:
2/(1/.58 + 1/.48)

0.5252830188679245

In [75]:
out = lstm.marginals(test)

In [76]:
out.shape

(4683,)

In [77]:
pred = np.zeros(out.shape)

In [79]:
pred[out >=.5] = 1

In [80]:
pred

array([ 1.,  1.,  0., ...,  0.,  0.,  1.])

In [83]:
true = L_gold_test.toarray().reshape(4683,)

In [85]:
true[true == -1] = 0

In [86]:
true

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [107]:
fp, tp, fn, tn = 0,0,0,0

In [108]:
for e,x in enumerate(true):
    if x==1 and random[e]==1:
        tp += 1
    elif x==1 and random[e]==0:
        fn += 1
    elif x==0 and random[e]==1:
        fp +=1
    elif x==0 and random[e]==0:
        tn +=1

In [109]:
fp + tp + fn + tn

4683

In [110]:
p = tp / (tp + fp)
r = tp / (tp + fn)

In [111]:
2 / (1/p + 1/r)

0.3996836277353019

In [93]:
# what i've shown here is just that this is the correct way to get the predictions.
np.savetxt('handlabeled_predictions.txt',pred)

In [94]:
np.savetxt('test.txt',true)

In [128]:
lstm.save(model_name='hand_labeled_small')

[LSTM] Model saved as <hand_labeled_small>


one potential next step: train GP classifier on the logits here and see if voting between the two gives better results at all - based on uncertainty?

However, this probably won't work because they are trained on the same data, so uncertainty should be the same for both.

Part of the problem here is that my hand labeled dataset and my snorkel dataset are trained on the same data-- there is no augmentation for the weak supervision model, so there's no "massive" amount of data that I can use. It would make things better if there had been augmentation in there as well -- then it's really a question of small vs large. 

In [104]:
random = np.random.random(pred.shape)

In [105]:
random[random < .5] = 0
random[random >=.5] = 1

In [106]:
random.shape

(4683,)

In [113]:
scores = []
for t in range (1000):
    random = np.random.random(pred.shape)
    random[random < .5] = 0
    random[random >=.5] = 1
    fp, tp, fn, tn = 0,0,0,0
    for e,x in enumerate(true):
        if x==1 and random[e]==1:
            tp += 1
        elif x==1 and random[e]==0:
            fn += 1
        elif x==0 and random[e]==1:
            fp +=1
        elif x==0 and random[e]==0:
            tn +=1
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    scores.append(2 / (1/p + 1/r))

In [114]:
scores = np.array(scores)

In [115]:
scores.mean()

0.39219376671965334